In [1]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch

In [7]:
# 訓練が終了したモデルとトークナイザを保存
# トークナイザとモデルの準備
tokenizer = T5Tokenizer.from_pretrained('JP28/t5-end2end-questions-generation', model_max_length=128)
model = T5ForConditionalGeneration.from_pretrained('JP28/t5-end2end-questions-generation')

model.save_pretrained("./output/model")
tokenizer.save_pretrained('./output/model')

OSError: Can't load tokenizer for 'JP28/t5-end2end-questions-generation'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'JP28/t5-end2end-questions-generation' is the correct path to a directory containing all relevant files for a T5Tokenizer tokenizer.

In [2]:
class WordTranslationDataset(Dataset):
    def __init__(self, tokenizer, df, max_length=128):
        self.tokenizer = tokenizer
        self.df = df
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        english_word = row['English']
        wrong_translation = row['Japanese']
        inputs = self.tokenizer.encode_plus(english_word, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        outputs = self.tokenizer.encode_plus(wrong_translation, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': outputs['input_ids'].flatten(),
        }

# データの読み込みと前処理
# 英単語と誤った日本語訳のペアのcsv
df = pd.read_csv('./input/en_and_ja.csv')
train_df, eval_df = train_test_split(df, test_size=0.2)

# トークナイザとモデルの準備
tokenizer = T5Tokenizer.from_pretrained('./output/model')
model = T5ForConditionalGeneration.from_pretrained('./output/model')

# データセットの作成
train_dataset = WordTranslationDataset(tokenizer, train_df)
eval_dataset = WordTranslationDataset(tokenizer, eval_df)

# トレーニングの設定
training_args = Seq2SeqTrainingArguments(
    #モデルのチェックポイントやトレーニングの結果などが保存されるディレクトリです。
    #異なる実験結果を区別するために、実行ごとに一意なディレクトリを指定することが重要です。
    output_dir='./output/results',  
    
    #トレーニングのエポック数を指定します。モデルがデータセット全体を何回トレーニングするかを制御します。
    #適切なエポック数は、データセットのサイズやタスクに依存します。
    #過剰なトレーニングは過学習を引き起こす可能性があります。
    num_train_epochs=3,
    
    #レーニングおよび評価時のデバイスごとのバッチサイズを指定します。
    #モデルとGPUメモリの制約に合わせて適切なバッチサイズを選択する必要があります。
    #メモリが不足する場合は、バッチサイズを減らす必要があります。
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=16,
    
    #オプティマイザのウォームアップステップ数を指定します。
    #ウォームアップステップは、学習率を線形に増加させるために使用されるステップ数です。
    #ウォームアップにより、初期の学習が安定化し、最適な学習率に到達するのを助けます。
    warmup_steps=500, 
    
    #重みの減衰（L2正則化）の係数を指定します。
    #重みの減衰は、過学習を防ぐために使用されます。値が大きいほど正則化の強度が高くなります。
    weight_decay=0.01,
    
    #学習率は、最適化アルゴリズムがモデルの重みを更新する際に使用するステップサイズを制御します。
    #学習率が小さすぎると収束が遅くなり、学習率が大きすぎると発散する可能性があります。
    #適切な学習率を選択することは、モデルの収束性と最終的な性能に重要な影響を与えます。
    learning_rate=5e-5,
    
    #勾配アキュムレーションは、バッチサイズを制限されたメモリリソースの中で増やすための手法です。
    #通常、バッチサイズを大きくすると、GPUメモリが不足する場合があります。
    #勾配アキュムレーションでは、複数の小さいバッチを逐次的に処理し、
    #その勾配を一定のステップ数ごとに累積することで、大きなバッチサイズの効果を得ることができます。
    #gradient_accumulation_stepsは、何ステップごとに勾配を累積するかを指定します。
    gradient_accumulation_steps=1,
    
    #ログファイルが保存されるディレクトリです。
    #トレーニングの進捗やメトリクスなどが記録されます。
    logging_dir='./output/logs',        
)

### GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    model.cuda()
    
# トレーナーの作成
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 訓練
trainer.train()


/Users/okumuradaichi/opt/anaconda3/envs/leanrning_word_env/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=57, training_loss=14.902842739172149, metrics={'train_runtime': 259.4881, 'train_samples_per_second': 0.879, 'train_steps_per_second': 0.22, 'total_flos': 8677649940480.0, 'train_loss': 14.902842739172149, 'epoch': 3.0})

In [ ]:
# 訓練が終了したモデルとトークナイザを保存
model.save_pretrained("./output/model")
tokenizer.save_pretrained('./output/model')

In [2]:
# トークナイザとモデルの準備
tokenizer = T5Tokenizer.from_pretrained('./output/model')
model = T5ForConditionalGeneration.from_pretrained('./output/model')

In [3]:
from Levenshtein import distance

In [4]:
# モデルが正しい日本語訳とどれだけ異なるかを評価する
def evaluate_model(model, tokenizer, df):
    model.eval()  # evaluation mode
    total_distance = 0

    for _, row in df.iterrows():
        english_word = row['English']
        correct_translation = row['Japanese']
        inputs = tokenizer.encode_plus(english_word, return_tensors='pt')
        outputs = model.generate(**inputs, max_new_tokens=128)
        generated_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

        total_distance += distance(correct_translation, generated_translation)
    
    average_distance = total_distance / len(df)
    return average_distance

eval_df = pd.read_csv('./input/en_and_ja.csv')
average_distance = evaluate_model(model, tokenizer, eval_df)
print(f"Average edit distance: {average_distance}")

<pad><extra_id_0>.<extra_id_1>.<extra_id_2>.<extra_id_3> selfishness<extra_id_4> selfishness<extra_id_5>selfishness. selfishness selfishness
<pad><extra_id_0>.<extra_id_1> incorporate<extra_id_2> incorporate into your company<extra_id_3>incorporate into your company incorporate into your company incorporate into
<pad><extra_id_0> quarryman<extra_id_1>quarryman, quarryman, quarryman, quarryman, quarryman, quarry
<pad><extra_id_0>.<extra_id_1> hobbyists<extra_id_2>hobbyists hobbyists hobbyists hobbyists hobby
<pad><extra_id_0>.<extra_id_1>.<extra_id_2>.<extra_id_3>.<extra_id_4>greetings from the greetings card. greetings from
<pad><extra_id_0>.<extra_id_1>.<extra_id_2>state of the art state of the art state of the art state of the
<pad><extra_id_0>condense condense condense condense condense condense con
<pad><extra_id_0>I can’t wait to see what you can do to help. I can’t wait
<pad><extra_id_0>.<extra_id_1>.<extra_id_2>. Connect. Connect. Connect. Connect. Connect. Connect. Connect.
<pa